In [0]:
import requests
import json
from datetime import datetime, timedelta
from typing import List, Dict
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, lit
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
%run /Shared/Nara/projeto_final/env

In [0]:
df = spark.read.format('delta').load('dbfs:/FileStore/harmonized')
df.display()

author,content,description,publishedAt,title,url,urlToImage,from_date,source_id,source_name,description_pt,content_pt,title_pt
null,O DNA de ursos polares que viveram entre 100 mil anos e 70 mil anos atrás indica que o material genético que hoje caracteriza a espécie ainda não estava totalmente presente nessa época. É como se os … [+3505 chars],Estudo aponta para possibilidade de boa parte do processo de adaptação desses bichos ao Ártico ter acontecido antes que pelagem totalmente branca fosse traço universal da espécie,2024-09-21T01:36:19Z,null,https://www1.folha.uol.com.br/ciencia/2024/09/ursos-polares-ainda-nao-eram-tao-polares-assim-ha-cerca-de-100-mil-anos.shtml,https://f.i.uol.com.br/fotografia/2024/09/17/172660206466e9db50d55f7_1726602064_3x2_rt.jpg,2024-09-20,null,Uol.com.br,Estudo aponta para possibilidade de boa parte do processo de adaptação desses bichos ao Ártico ter acontecido antes que pelagem totalmente branca fosse traço universal da espécie,O DNA de ursos polares que viveram entre 100 mil anos e 70 mil anos atrás indica que o material genético que hoje caracteriza a espécie ainda não estava totalmente presente nessa época. É como se os … [+3505 chars],null
null,"Captive tigers in the United States outnumber those living in the wild. The World Wildlife Federation estimates around 5,000 of the big cats reside in the U.S., mostly owned by private citizens. The… [+8507 chars]","Using DNA from rescued captive tigers in the U.S., Stanford researchers have developed a genetic reference panel that can help identify tigers suitable for conservation and prosecute animal abusers.",2024-09-19T00:00:00Z,Can captive tigers be part of the effort to save wild populations?,https://news.stanford.edu/stories/2024/09/can-captive-tigers-be-part-of-the-effort-to-save-wild-populations,https://news.stanford.edu/__data/assets/image/0036/159597/Nyla_Blanca_InSyncExotics.jpg,2024-09-19,null,Stanford.edu,"Usando DNA de tigres resgatados em cativeiro nos EUA, pesquisadores de Stanford desenvolveram um painel de referência genética que pode ajudar a identificar tigres adequados para conservação e processar abusadores de animais.","Tigres em cativeiro nos Estados Unidos superam em número aqueles que vivem na natureza. A World Wildlife Federation estima que cerca de 5.000 desses grandes felinos residem nos EUA, a maioria de propriedade de cidadãos privados. O… [+8507 caracteres]",Tigres em cativeiro podem fazer parte do esforço para salvar populações selvagens?
null,"Genome-wide association studies have great potential for advancing our understanding of the genetic background of diseases, but so far, few association studies have focused on maternal and newborn di… [+2729 chars]","Genome-wide association studies have great potential for advancing our understanding of the genetic background of diseases, but so far, few association studies have focused on maternal and newborn diseases, and most have been based on predominantly European p…",2024-10-09T16:15:44Z,"Genetic analysis of 25,000 Chinese mothers and infants reveals unique genes associated with disease risk",https://www.sciencedaily.com/releases/2024/10/241009121544.htm,https://www.sciencedaily.com/images/scidaily-icon.png,2024-10-09,null,Science Daily,"Estudos de associação em todo o genoma têm grande potencial para avançar nossa compreensão do contexto genético das doenças, mas até agora, poucos estudos de associação se concentraram em doenças maternas e neonatais, e a maioria foi baseada em pesquisas predominantemente europeias.","Estudos de associação em todo o genoma têm grande potencial para avançar nossa compreensão do contexto genético das doenças, mas até agora, poucos estudos de associação se concentraram em doenças maternas e neonatais… [+2729 caracteres]",Análise genética de 25.000 mães e bebês chineses revela genes únicos associados ao risco de doenças
null,"In 1898, two male lions terrorized an encampment of bridge builders on the Tsavo River in Kenya. The lions, which wer

In [0]:
!pip install --upgrade pip 
!pip install --upgrade Pillow
!pip install wordcloud
!pip install nltk

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql import functions as F
from pyspark.sql.functions import regexp_replace
import nltk

nltk.download("stopwords")

tokenizer = Tokenizer(inputCol="content_pt", outputCol="tokens")
df_tokens = tokenizer.transform(df)

stopwordList = nltk.corpus.stopwords.words('portuguese')
remover = StopWordsRemover(inputCol='tokens', outputCol="filtered_tokens", stopWords=stopwordList)
df_clean = remover.transform(df_tokens)

df_clean = df_clean.withColumn("filtered_tokens", 
    F.expr("filter(filtered_tokens, x -> x rlike '^[a-zA-Z]+$')"))

df_exploded = df_clean.withColumn("word", F.explode(F.col("filtered_tokens")))

df_word_count = df_exploded.groupBy("word").count().orderBy(F.desc("count"))

top_words = df_word_count.limit(100).collect()

df_word_count.display()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


word,count
dna,16
dados,11
empresa,9
sequenciamento,6
outubro,6
global,5
sobre,5
duas,4
grandes,4
homo,4


Databricks visualization. Run in Databricks to view.